In [1]:
import torch
import torch.nn as nn    
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
X,y = load_iris().data, load_iris().target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for i,(inputs,labels) in enumerate(train_loader):
    print(f"{inputs.shape}------{labels.shape}")
    


torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([8, 4])------torch.Size([8])


In [3]:
class IrisNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(4,3)
    def forward(self,x):
        x = self.fc1(x)
        return x

In [4]:
model = IrisNet()
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

In [5]:
epochs = 100
correct = 0
total = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        _,predicted = torch.max(outputs.data,1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        train_acc = correct/total * 100
        
        loss = loss_fcn(outputs,labels)
        
        loss.backward()
        optimizer.step()
        running_loss+= loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{epochs}], | Loss: {epoch_loss:.4f}, | training_accuracy: {train_acc:.2f}%')

Epoch [1/100], | Loss: 1.1238, | training_accuracy: 38.33%
Epoch [2/100], | Loss: 0.8101, | training_accuracy: 50.83%
Epoch [3/100], | Loss: 0.5780, | training_accuracy: 62.22%
Epoch [4/100], | Loss: 0.4773, | training_accuracy: 68.33%
Epoch [5/100], | Loss: 0.4281, | training_accuracy: 71.67%
Epoch [6/100], | Loss: 0.4011, | training_accuracy: 73.89%
Epoch [7/100], | Loss: 0.3818, | training_accuracy: 75.60%
Epoch [8/100], | Loss: 0.3679, | training_accuracy: 76.98%
Epoch [9/100], | Loss: 0.3569, | training_accuracy: 77.96%
Epoch [10/100], | Loss: 0.3462, | training_accuracy: 78.83%
Epoch [11/100], | Loss: 0.3372, | training_accuracy: 79.77%
Epoch [12/100], | Loss: 0.3288, | training_accuracy: 80.62%
Epoch [13/100], | Loss: 0.3212, | training_accuracy: 81.41%
Epoch [14/100], | Loss: 0.3164, | training_accuracy: 82.08%
Epoch [15/100], | Loss: 0.3074, | training_accuracy: 82.72%
Epoch [16/100], | Loss: 0.3025, | training_accuracy: 83.28%
Epoch [17/100], | Loss: 0.2965, | training_accura

In [6]:
torch.max(outputs)

tensor(7.8555, grad_fn=<MaxBackward1>)

In [9]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _ , predicted = torch.max(outputs.data, 1)  # Get the index of the max logit
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
acc = (correct / total) * 100  
print(f'Test Accuracy: {acc:.2f}%')



Test Accuracy: 100.00%


In [8]:
from helper_function import plot_decision_boundary